# Segmentation of selected solar corona structures

# <font color="red">Evaluation and testing only: Segmentation of Coronal Holes


---


### Author: Bc. Ľubomír Lazor
### Supervisor: doc. Ing. Martin Sarnovský, PhD.
Department of Cybernetics and Artificial Intelligence, Faculty of Electrical Engineering and Informatics, Technical University of Košice, Košice, Slovakia


## Abstract
This thesis explores the use of deep learning for the automated segmentation of coronal holes (CHs) and active regions (ARs) in solar EUV images. These structures are key to understanding solar activity and predicting space weather. Building on the SCSS-Net model, a U-Net-based architecture, this work combines theoretical insights with practical modeling to assess performance of various configurations. The results show that deep learning can offer a reliable, scalable solution for the segmentation of solar structures driven by thoughtful data handling and model design.

---

### Disclaimer: this code is based on SCSS-Net

Overview of the notebook:
1. Environment set-up
2. The only things that you need to change
3. Evaluation
4. (OPTIONAL) Custom threshold for binary predictions
5. Visualization
6. Reiss dataset visualization

# 1. Enviroment set-up

* Set-up the environment

In [ ]:
import sys, os
!pip install -r requirements.txt
sys.path.append('../scss-net/src')

* Import libraries

In [ ]:
import glob
from PIL import Image
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

* Import local utilities

In [ ]:
from src.model_scss_net import scss_net
from src.metrics import dice_np, iou_np, dice, iou
from src.utils import plot_imgs, plot_metrics

# 2. The only things that you need to change
test_images/masks_path - the directory of the images and masks you want to use for testing
<br>mode_path - the directory of the model you want to use 

(it has to be a full model, not just weights, check "5. Extra tips (potential fixes)" in "U-net_CH.ipynb" for more information

In [ ]:
# Paths relative to scss-net folder, if they are outside of it, they have to be defined from the root directory
# Load images
test_images_path = "/data/CH/images_renamed/test/*.png"
test_masks_path = "/data/CH/masks_renamed/test/*.png"

# Load model
model_path = 'models/ch_model.h5'

* Prepare test set(load, resize, convert, normalize and reshape testing data)

In [ ]:
IMG_SIZE = 256  # Resize images to 256x256

imgs_test = sorted(glob.glob(test_images_path))
masks_test = sorted(glob.glob(test_masks_path))

print(f"Imgs number = {len(imgs_test)}\nMasks number = {len(masks_test)}")

# Convert images to numpy array
imgs_test_list = []
masks_test_list = []
for image, mask in zip(imgs_test, masks_test):
    imgs_test_list.append(np.array(Image.open(image).convert("L").resize((IMG_SIZE, IMG_SIZE))))
    masks_test_list.append(np.array(Image.open(mask).convert("L").resize((IMG_SIZE, IMG_SIZE))))

# Normalization from (0; 255) to (0; 1)
x_test = np.asarray(imgs_test_list, dtype=np.float32)/255
y_test = np.asarray(masks_test_list, dtype=np.float32)/255

# Reshape to (n_imgs, height, width, channels)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1], y_test.shape[2], 1)

* Load the pre-trained model and its metrics

In [ ]:
# Define the custom objects (iou and dice)
custom_objects = {
    'iou': iou,
    'dice': dice
}

# Load the model and pass the custom objects
model = load_model(model_path, custom_objects=custom_objects)

# Check the model summary to confirm it's loaded correctly
#print(model.summary())

## 3. Evaluation

* Predict new masks for test set

In [ ]:
y_pred = model.predict(x_test)

* Calculate the best threshold value

In [ ]:
# Find the best threshold and save it to global variables

best_threshold = 0
best_dice = 0

for threshold in np.arange(0, 1.05, 0.05):
    y_pred_bin = np.where(y_pred > threshold, 1, 0)
    
    dice_tresh = np.round(dice_np(y_test, y_pred_bin), 4)
    iou_tresh = np.round(iou_np(y_test, y_pred_bin), 4)
    
    if dice_tresh > best_dice:
        best_dice = dice_tresh
        best_threshold = threshold
        best_iou = iou_tresh
    print(f"Threshold: {round(threshold,2)} | Dice Score: {dice_tresh} | IoU Score: {iou_tresh}")

print(f"Best Threshold: {round(best_threshold,2)} with Dice Score: {best_dice} and IoU Score: {best_iou}")

* Create a binary prediction based on the best theshold

In [ ]:
# Binary prediction with the best threshold
y_pred_bin = np.where(y_pred > best_threshold, 1, 0)

dice = np.round(dice_np(y_test, y_pred), 4)
iou_test = np.round(iou_np(y_test, y_pred), 4)

dice_tresh = np.round(dice_np(y_test, y_pred_bin), 4)
iou_test_tresh = np.round(iou_np(y_test, y_pred_bin), 4)

print(f"Test:\nDice: {dice} Dice_tresh: {dice_tresh}\n IoU: {iou_test} IoU_tresh: {iou_test_tresh}\n")

## 4. (OPTIONAL) Custom threshold for binary predictions

In case you want to try a custom threshold, you can do so here

In [ ]:
# Set a global variable threshold to desired value between 0 and 1
threshold = 0.01

# Create binary prediction
y_pred_bin = np.where(y_pred > threshold, 1, 0)
    
dice_tresh = np.round(dice_np(y_test, y_pred_bin), 4)
iou_tresh = np.round(iou_np(y_test, y_pred_bin), 4)
    

print(f"Threshold: {round(threshold,2)} | Dice Score: {dice_tresh} | IoU Score: {iou_tresh}")

## 5. Visualization

* Function designed to show random images from the test set wihtout metrics

* Visualization of the predicted segmentation: 
 - Image: input image 
 - Mask: ground truth (annotation mask)
 - Prediction: output of the model (predicted segmentation mask)
 - Overlay: predicted segmentation mask overlaid on the input image

In [ ]:
import random

def plot_no_metrics_random(imgs, masks, predictions, n_imgs=10, seed=None):
    """
    Plots a random subset of images, masks, and predictions, with optional reproducible randomness.
    
    :param numpy.array imgs: array of images
    :param numpy.array masks: array of masks
    :param numpy.array predictions: array of predictions
    :param int n_imgs: number of images to plot
    :param int seed: (Optional) Random seed for reproducibility
    :return: matplotlib.pyplot object
    """
    if seed is not None:
        random.seed(seed)  # Set seed for reproducibility

    total_imgs = len(imgs)
    n_imgs = min(n_imgs, total_imgs)  # Ensure we don't exceed available images

    # Randomly select `n_imgs` indices
    random_indices = random.sample(range(total_imgs), n_imgs)
    
    fig, axs = plt.subplots(n_imgs, 4, figsize=(16, n_imgs * 4), squeeze=False)

    for row_idx, img_idx in enumerate(random_indices):  
        img = imgs[img_idx].squeeze()  
        mask = masks[img_idx].squeeze()
        pred = predictions[img_idx].squeeze()

        # Create red overlay (RGBA: Red, Green, Blue, Alpha)
        red_overlay = np.zeros((*pred.shape, 4))  
        red_overlay[..., 0] = pred  # Red channel
        red_overlay[..., 3] = pred  # Alpha (opacity based on prediction)

        # Set titles (only on the first row)
        if row_idx == 0:
            axs[0, 0].set_title("Image", fontsize=15)
            axs[0, 1].set_title("SPoCA Segmentation", fontsize=15)
            axs[0, 2].set_title("U-Net prediction", fontsize=15)
            axs[0, 3].set_title("Overlay", fontsize=15)

        # Show images
        axs[row_idx, 0].imshow(img, cmap="gray", interpolation=None)
        axs[row_idx, 1].imshow(mask, cmap="gray", interpolation=None)
        axs[row_idx, 2].imshow(pred, cmap="gray", interpolation=None)
        axs[row_idx, 3].imshow(img, cmap="gray", interpolation=None)  # Background
        axs[row_idx, 3].imshow(red_overlay)  # Overlay red mask on top

        # Hide axes
        for j in range(4):
            axs[row_idx, j].axis("off")
    
    plt.tight_layout()
    return plt

* Normal segmentation prediction visualization

In [ ]:
plot_no_metrics_random(imgs=x_test, masks=y_test, predictions=y_pred, n_imgs=10, seed=42).show()

* Binary segmentation prediction visualization

In [ ]:
y_pred_b = model.predict(x_test)
plot_no_metrics_random(imgs=x_test, masks=y_test, predictions=y_pred_bin, n_imgs=10, seed=42).show()

# 6. Reiss dataset visualization
* This section is designed to compare the models with the masks from CHRONNOS on images from dataset created by Reiss et. al. (2024)

* Custom function to plot all 29 images along with masks

* Visualization of the predicted segmentation: 
 - Image: input image 
 - Mask: ground truth (CHRONNOS annotation mask)
 - Prediction: output of the model (predicted segmentation mask)
 - Overlay: predicted segmentation mask overlaid on the input image

In [ ]:
def plot_no_metrics(imgs, masks, predictions, n_imgs=29):
    """
    Plots images, masks, prediction masks, and overlays side by side.
    
    :param numpy.array imgs: array of images
    :param numpy.array masks: array of CHRONNOS masks
    :param numpy.array predictions: array of predictions
    :param int n_imgs: number of images to plot
    :return: matplotlib.pyplot object
    """
    n_imgs = min(n_imgs, len(imgs))  # Ensure we don't exceed available images
    
    fig, axs = plt.subplots(n_imgs, 4, figsize=(16, n_imgs * 4), squeeze=False)
    
    for i in range(n_imgs):
        img = imgs[i].squeeze()  # Convert (H, W, 1) → (H, W)
        mask = masks[i].squeeze()
        pred = predictions[i].squeeze()

        # Create red overlay (RGBA: Red, Green, Blue, Alpha)
        red_overlay = np.zeros((*pred.shape, 4))  # (H, W, 4) for RGBA
        red_overlay[..., 0] = pred  # Red channel
        red_overlay[..., 3] = pred  # Alpha channel (opacity based on prediction)

        # Set titles (only on the first row)
        if i == 0:
            axs[0, 0].set_title("Image", fontsize=15)
            axs[0, 1].set_title("CHRONNOS segmentation", fontsize=15)
            axs[0, 2].set_title("U-Net prediction", fontsize=15)
            axs[0, 3].set_title("Overlay", fontsize=15)

        # Show images
        axs[i, 0].imshow(img, cmap="gray", interpolation=None)
        axs[i, 1].imshow(mask, cmap="gray", interpolation=None)
        axs[i, 2].imshow(pred, cmap="gray", interpolation=None)
        axs[i, 3].imshow(img, cmap="gray", interpolation=None)  # Background
        axs[i, 3].imshow(red_overlay)  # Overlay red mask on top

        # Hide axes
        for j in range(4):
            axs[i, j].axis("off")
    
    plt.tight_layout()
    return plt

* Prepare Reiss dataset for our use(load, resize, convert, normalize and reshape testing data)

In [ ]:
imgs_test = sorted(glob.glob("Reiss_dataset/images/*.png"))
masks_test = sorted(glob.glob("Reiss_dataset/masks/*.png"))

print(f"Imgs number = {len(imgs_test)}\nMasks number = {len(masks_test)}")

# Load data and convert images to numpy array
imgs_test_list = []
masks_test_list = []
for image, mask in zip(imgs_test, masks_test):
    imgs_test_list.append(np.array(Image.open(image).convert("L").resize((IMG_SIZE, IMG_SIZE))))
    masks_test_list.append(np.array(Image.open(mask).convert("L").resize((IMG_SIZE, IMG_SIZE))))

# Normalization from (0; 255) to (0; 1)
x_test = np.asarray(imgs_test_list, dtype=np.float32)/255
y_test = np.asarray(masks_test_list, dtype=np.float32)/255

# Reshape to (n_imgs, height, width, channels)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1], y_test.shape[2], 1)

* Load the pre-trained model and its metrics

In [ ]:
# Define the custom objects (iou and dice)
custom_objects = {
    'iou': iou,
    'dice': dice
}

# Load the model and pass the custom objects
model = load_model('models/final/ch_model.h5', custom_objects=custom_objects)

# Check the model summary to confirm it's loaded correctly
#print(model.summary())

* Predict new masks for test set

In [ ]:
y_pred = model.predict(x_test)

* Calculate the best threshold value

In [ ]:
best_threshold = 0
best_dice = 0

for threshold in np.arange(0, 1.05, 0.05):
    y_pred_bin = np.where(y_pred > threshold, 1, 0)
    
    dice_tresh = np.round(dice_np(y_test, y_pred_bin), 4)
    iou_tresh = np.round(iou_np(y_test, y_pred_bin), 4)
    
    if dice_tresh > best_dice:
        best_dice = dice_tresh
        best_threshold = threshold
        best_iou = iou_tresh
    print(f"Threshold: {round(threshold,2)} | Dice Score: {dice_tresh} | IoU Score: {iou_tresh}")

print(f"Best Threshold: {round(best_threshold,2)} with Dice Score: {best_dice} and IoU Score: {best_iou}")

* Create a binary prediction based on the best theshold

In [ ]:
y_pred_bin = np.where(y_pred > best_threshold, 1, 0)

dice = np.round(dice_np(y_test, y_pred), 4)
iou_test = np.round(iou_np(y_test, y_pred), 4)

dice_tresh = np.round(dice_np(y_test, y_pred_bin), 4)
iou_test_tresh = np.round(iou_np(y_test, y_pred_bin), 4)

print(f"Test:\nDice: {dice} Dice_tresh: {dice_tresh}\n IoU: {iou_test} IoU_tresh: {iou_test_tresh}\n")

### Custom threshold
* Create a binary prediction with a custom threshold

In [ ]:
y_pred_bin = np.where(y_pred > 0.01, 1, 0)  # Set threshold for predicted values

dice = np.round(dice_np(y_test, y_pred), 4)
iou_test = np.round(iou_np(y_test, y_pred), 4)

dice_tresh = np.round(dice_np(y_test, y_pred_bin), 4)
iou_test_tresh = np.round(iou_np(y_test, y_pred_bin), 4)

print(f"Test:\nDice: {dice} Dice_tresh: {dice_tresh}\n IoU: {iou_test} IoU_tresh: {iou_test_tresh}\n")

* Binary segmentation prediction visualization on the whole Reiss dataset

In [ ]:
y_pred_b = model.predict(x_test)  
plot_no_metrics(imgs=x_test, masks=y_test, predictions=y_pred_bin, n_imgs=29).show()